In [121]:
#Load necessary packages
import pandas as pd
import os
import numpy as np
import copy

In [249]:
#Load data created in Library_Week1_Notebook2_DataCleanUp

#Load data
os.chdir("//Users/Olga/Documents/INSIGHT2019/Library data/AllPldData")
Files_in_folder = os.listdir()
File_names = ['_2014', '_2015', '_2016', '_2001', '_2002', '_2003', 
              '_2004','_2005', '_2006', '_2007', '_2008', '_2009', 
              '_2010', '_2011', '_2012', '_2013', '_2000', '_1992', 
              '_1993', '_1994', '_1995', '_1996', '_1997', '_1998', '_1999']
Files = []

for filename in os.listdir():
    if filename.endswith('csv'):
        Files.append(pd.read_csv(filename, encoding = 'latin-1', low_memory = False))
        
#create dictionary of libraries labeled by year
LibData_dict = {}
for i in range(0, len(File_names)):
    LibData_dict[File_names[i]] = Files[i]
    
for k, v in LibData_dict.items():
    print(k, v.shape, type(LibData_dict[k]))

ColumnsPresent = pd.DataFrame(index=range(0),columns=range(0))

for k, v in LibData_dict.items():
    dataframe_to_join = pd.DataFrame({k: list(v.columns)})
    ColumnsPresent = ColumnsPresent.join(dataframe_to_join, how = 'outer')

#Rearrange columns in dataframe. 
cols = ColumnsPresent.columns.tolist()
cols_rearr = cols[17:] + cols[0:17]
cols_rearr2 = cols_rearr[0:8] + cols_rearr[24:25] + cols_rearr[11:24] + cols_rearr[8:11]

ColumnsPresent_rearr = ColumnsPresent[cols_rearr2]

ColumnsPresent_rearr_str = ColumnsPresent_rearr.applymap(str)
ColumnsPresent_rearr_str = ColumnsPresent_rearr_str.reset_index(drop = True)

#create DF of unique column values
UniqueLabels = np.unique(ColumnsPresent_rearr_str.values)
LabelsBoolDF = pd.DataFrame(UniqueLabels)

#Create DF w boolean values for prensence of each unique value
#THIS IS THE END GOAL OF THIS DATA
for j in range(len(ColumnsPresent_rearr_str.columns)):
    lis = []
    for i in range(len(UniqueLabels)):
        lis.append(ColumnsPresent_rearr_str.iloc[:, j].isin([UniqueLabels[i]]).any())
    LabelsBoolDF[ColumnsPresent_rearr_str.columns[j]] = lis
    
## Sum all TRUE values. Value of 25 = all true
sum_list = []
for i in range(LabelsBoolDF.shape[0]):
    sum_list.append(sum(LabelsBoolDF.iloc[i, 1:26]))

#Label OK in Notes
LabelsBoolDF['TRUE SUM'] = sum_list
LabelsBoolDF['Notes'] = ""
LabelsBoolDF.loc[LabelsBoolDF[LabelsBoolDF['TRUE SUM']==25].index, 'Notes'] = "OK"

#rows in which not all TRUE (meaning this column/data does not exist for all year)
MissingData = LabelsBoolDF[LabelsBoolDF['TRUE SUM']!=25].index.tolist()

#Columns in which data is unavailable for all years.
LabelsBoolDF.iloc[1,27] = "OK - Mailing address. Not necessary for analysis."
LabelsBoolDF.iloc[2,27] = "OK - Address type. Not necessary for analysis."
LabelsBoolDF.iloc[3,27] = "ATTEND was called VISITS after 1998. Combine with VISITS."
LabelsBoolDF.iloc[4:7,27] = "AUDIO is divided into AUDIO_PH, AUDIO_DL starting 2010. Source: 2010 Documentation."
LabelsBoolDF.iloc[12,27] = "Capital revenue, only computed after 2003. No equivalent prior, it seems."
LabelsBoolDF.iloc[13,27] = "OK - Call Based Statistical Area. Not necessary for analysis"
LabelsBoolDF.iloc[14,27] = "OK - Congressional district code. Not necessary for analysis."
LabelsBoolDF.iloc[15,27] = "OK - Census block. Not necessary for analysis."
LabelsBoolDF.iloc[17,27] = "OK - Census track. Not necessary for analysis."
LabelsBoolDF.iloc[19,27] = "OK - CITY_M. Not necessary for analysis."
LabelsBoolDF.iloc[21,27] = "OK - county code. Not necessary for analysis."
LabelsBoolDF.iloc[22,27] = "County population. Incomplete. Probably OK. Just need to make sure the there is another better measure of population - POPU_LSA I think."
LabelsBoolDF.iloc[24,27] = "OK - FSCS Public Library Definition. Not necessary for analysis."
LabelsBoolDF.iloc[25,27] = "C_LEGBAS - Legal Basis Code. SAME as C_LEGBASE 1998 and ON. Combine the two rows."
LabelsBoolDF.iloc[26,27] = "C_LEGBASE - Legal Basis Code. SAME as C_LEGBAS 1997 and prior. Combine the two rows."
LabelsBoolDF.iloc[28,27] = "DATABASE - Total licensed databases."
LabelsBoolDF.iloc[29,27] = "DB_LOC - Local licensed databases"
LabelsBoolDF.iloc[30,27] = "DB_LO_OT - Local/other databases"
LabelsBoolDF.iloc[31:33,27] = "More database info"
LabelsBoolDF.iloc[33,27] = "DUPLI. Called HRS_OPEN 1998 ON. Combine with HRS_OPEN."
LabelsBoolDF.iloc[34,27] = "EBOOK. Starts 2003 (makes sense)"
LabelsBoolDF.iloc[35,27] = "EC_LO_OT. Local/other electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[36,27] = "EC_ST. State electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[37,27] = "ELACCEXP. Operating expendature for electronic access."
LabelsBoolDF.iloc[38,27] = "ELCONT. 2016 only [2016 doc: first collection]. Electronic content use. Total circulation."
LabelsBoolDF.iloc[39,27] = "ELECCOLL. Total electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[40,27] = "ELINFO. Retrival of electronic infomation. 2016 only. "
LabelsBoolDF.iloc[41,27] = "ELMATCIRC. Circulation of electronic material. 2013-2016."
LabelsBoolDF.iloc[42,27] = "ELMATEXP. 1995-2016. Circulation of electronic materials. This is a good one to use!"
LabelsBoolDF.iloc[43,27] = "ELMATS. Number of library materials in electronic format. 1995-2002"
LabelsBoolDF.iloc[44,27] = "ELSVCACC. Library access to electronic services. Yes, no. Unnecessary. 1995-2002."
LabelsBoolDF.iloc[45,27] = "OK - ENDDATE. Not necessary for analysis."
LabelsBoolDF.iloc[46,27] = "ERES_USR. 1999-2005. Electronic resource users per year. "
LabelsBoolDF.iloc[47,27] = "ESUBSCRP. 2003-2009. Electronic subscriptions."
LabelsBoolDF.iloc[48,27] = "FCAP_REV. Federal gov capital revenue. 2005 and on."
LabelsBoolDF.iloc[50:61,27] = "OK - FILL. Not necessary for analysis."
LabelsBoolDF.iloc[61,27] = "OK - county code. Not necessary for analysis."
LabelsBoolDF.iloc[62,27] = "OK - place code. Not necessary for analysis."
LabelsBoolDF.iloc[63,27] = "OK - state code. Not necessary for analysis."
LabelsBoolDF.iloc[65:133,27] = "OK - imputation flags. Not necessary for analysis."
LabelsBoolDF.iloc[133,27] = "OK - GAL. Not necessary for analysis. Geocode location."
LabelsBoolDF.iloc[134,27] = "OK - GALMS. Not necessary for analysis."
LabelsBoolDF.iloc[135,27] = "GEOCODE - Geographical code. Might be useful. Can add to old datasets that don't have it."
LabelsBoolDF.iloc[136,27] = "OK - ID. Not necessary."
LabelsBoolDF.iloc[137,27] = "GPTERMS. Internet computers used by general public. 1998-2016."
LabelsBoolDF.iloc[138,27] = "HRS_OPEN. Called DUPLI 1997 and prior. Combine with DUPLI."
LabelsBoolDF.iloc[139:181,27] = "OK - imputation flags. Not necessary for analysis."
LabelsBoolDF.iloc[181,27] = "OK - County code. Not necassary for analysis."
LabelsBoolDF.iloc[182,27] = "OK - code. Not necassary for analysis."
LabelsBoolDF.iloc[183,27] = "INETACC - Internet access."
LabelsBoolDF.iloc[184,27] = "OK - imputation flag"
LabelsBoolDF.iloc[185,27] = "Kid program attendance. Combine with KIDATTEN."
LabelsBoolDF.iloc[186,27] = "Kid program attendance. Combine with KIDATTEND."
LabelsBoolDF.iloc[188,27] = "KIDPRO - Total kids program attendance."
LabelsBoolDF.iloc[189,27] = "OK - LATITUDE. NOt necessary for analysis."
LabelsBoolDF.iloc[190,27] = "LCAP_REV. Local gov capital revenue."
LabelsBoolDF.iloc[193,27] = "LIBRARIAN. Combine with LIBRARIA."
LabelsBoolDF.iloc[194,27] = "LIBRARIA. Combine with LIBRARIAN."
LabelsBoolDF.iloc[197,27] = "Urban-centric locale code. 2008-2016. 2008 doc: started in 2008. Can add to older years if necessary."
LabelsBoolDF.iloc[199,27] = "OK - LOGITUDE. Not necessary for analysis."
LabelsBoolDF.iloc[200,27] = "OK - LSABOUND. Legal services boundary. Not necessary for analysis."
LabelsBoolDF.iloc[202,27] = "OK - MAT_CENT. Match centroid. Not necessary for analysis."
LabelsBoolDF.iloc[203,27] = "OK - MAT_TYPE. Match status code. Not necessary for analysis."
LabelsBoolDF.iloc[204,27] = "OK - MICROF. Metro, micropolitan. Not necessary for analysis."
LabelsBoolDF.iloc[205,27] = "OK - MSTATUS. Match status. Not necessary for analysis." 
LabelsBoolDF.iloc[207,27] = "OCAP_REV. Other capital revenue."
LabelsBoolDF.iloc[209,27] = "OTHMATEX. Operating expenditures for all other library materials."
LabelsBoolDF.iloc[213,27] = "PHYSCIR. First collected in 2016."
LabelsBoolDF.iloc[214,27] = "PITUSR. Public use of internet."
LabelsBoolDF.iloc[215,27] = "POPU. Combine with POPU_LSA."
LabelsBoolDF.iloc[216,27] = "POPU_LSA. Population of legal service area. Combine with POPU."
LabelsBoolDF.iloc[217,27] = "POPU_UND. Combine with POPU_UNDUP."
LabelsBoolDF.iloc[218,27] = "POPU_UNDUP. Combine with POPU_UND."
LabelsBoolDF.iloc[219,27] = "OK - POSTMS. Postal match status. Unnecessary for analysis."
LabelsBoolDF.iloc[220,27] = "PRIMATEXP - Print material expenditures."
LabelsBoolDF.iloc[221,27] = "PUB_FIPS - OK. State code. Not necessary for analysis."
LabelsBoolDF.iloc[222,27] = "REAPLOCALE - urbancentric location code. Only 2016. Anything similar used before?"
LabelsBoolDF.iloc[223,27] = "REFERENC. Combine with REFERENCE."
LabelsBoolDF.iloc[224,27] = "REFERENCE. Combine with REFERENC."
LabelsBoolDF.iloc[225,27] = "REGBOR. Registered users. Anything similar prior to 2006?"
LabelsBoolDF.iloc[228,27] = "SCAP_REV. State revenue."
LabelsBoolDF.iloc[229,27] = "OK - SCORE. Address match. Not necessary for analysis."
LabelsBoolDF.iloc[231,27] = "STAFFEXP - Total staff exp. Salary and benefits. Combine with TOTEXP."
LabelsBoolDF.iloc[232,27] = "OK - STARTDAT. Reporting period start date. Not necessary for analysis."
LabelsBoolDF.iloc[233,27] = "OK - STATADDR. Address change code. Not necessary for analysis."
LabelsBoolDF.iloc[234,27] = "OK - STATNAME. Name change code. Not necessary for analysis."
LabelsBoolDF.iloc[235,27] = "OK - STATSTRU. Structure change code. Not necessary for analysis."
LabelsBoolDF.iloc[236,27] = "OK - STFTERMS. Internal terminals used by staff only. Not necessary for analysis."
LabelsBoolDF.iloc[238,27] = "SUBSCRIP. Combine with SUBSCRIPT."
LabelsBoolDF.iloc[239,27] = "SUBSCRIPT. Combine with SUBSCRIP."
LabelsBoolDF.iloc[240,27] = "TOTATTEN. Total attendance at library programs."
LabelsBoolDF.iloc[242,27] = "TOTCOLL. Total collection use. Only 2016."
LabelsBoolDF.iloc[243,27] = "TOTEXP. TOtal staff expenditures. Salary and benefits. Combine with STAFFEXP."
LabelsBoolDF.iloc[244,27] = "TOTEXPCO. Total expenditure of lib collection. Combine with TOTEXPCOL."
LabelsBoolDF.iloc[245,27] = "TOTEXPCOL. Total expenditure of lib collection. Combine with TOTEXPCO."
LabelsBoolDF.iloc[247,27] = "TOTOPEXP. Total operating expenditures. Combine with TOTOPEXP1."
LabelsBoolDF.iloc[248,27] = "TOTOPEXP1. Total operating expenditures. Combine with TOTOPEXP."
LabelsBoolDF.iloc[249,27] = "TOTPEMP - Total paid FTE employees. Combine with TOTSTAFF."
LabelsBoolDF.iloc[250,27] = "TOTPRO - Total library programs. Can this be combined with another label?"
LabelsBoolDF.iloc[251,27] = "TOTSTAFF. Total paid FTE employees. Combine with TOTPEMP."
LabelsBoolDF.iloc[252:255,27] = "VIDEO was later split into VIDEO_DL and VIDEO_PH."
LabelsBoolDF.iloc[255,27] = "VISITS. Combine with ATTEND."
LabelsBoolDF.iloc[256,27] = "OK - WEB_ADDR. Not necessary for data."
LabelsBoolDF.iloc[257,27] = "WIFISESS. Any similar metric prior?"
LabelsBoolDF.iloc[258,27] = "YAATTEN. Young adult program attendance."
LabelsBoolDF.iloc[259,27] = "YAPRO. Total young adult programs."
LabelsBoolDF.iloc[260:262,27] = "OK - Submission year.  YR and YR_SUB are the same. Not necessary for analysis."
LabelsBoolDF.iloc[260:262,27] = "OK - Submission year.  YR and YR_SUB are the same. Not necessary for analysis."
LabelsBoolDF.iloc[262,27] = "ZIP. Combine with ZIP1."
LabelsBoolDF.iloc[263,27] = "ZIP1. Combine with ZIP."
LabelsBoolDF.iloc[264:268,27] = "OK - More zip information. Not necessary for analysis."
LabelsBoolDF.iloc[268,27] = "Note that lack of nan."

#Column names to keep
#LabelsBoolDF.loc[[0, 3:13, 15], 0].tolist()
Cols_to_keep_idx = [0]
Cols_to_keep_idx.extend(range(3,13))
Cols_to_keep_idx.extend((16, 18, 20, 25, 26))
Cols_to_keep_idx.extend(range(33, 49))
Cols_to_keep_idx.extend((64, 135, 137, 138, 183))
Cols_to_keep_idx.extend(range(185, 189))
Cols_to_keep_idx.extend(range(191, 199))
Cols_to_keep_idx.append(201)
Cols_to_keep_idx.extend(range(206, 219))
Cols_to_keep_idx.append(220)
Cols_to_keep_idx.extend(range(222, 229))
Cols_to_keep_idx.extend((230, 231, 237))
Cols_to_keep_idx.extend(range(237, 256))
Cols_to_keep_idx.extend(range(257, 260))
Cols_to_keep_idx.extend((262, 263))

Cols_to_keep_in_lib_dfs = LabelsBoolDF.iloc[Cols_to_keep_idx, 0].tolist()

#Make dictionaries of smaller dataframe with Cols_to_keep_in_lib_dfs ONLY
LibData_dict_reduced = dict()

for k, v in LibData_dict.items():
    LibData_dict_reduced[k] = pd.DataFrame()
    for i in Cols_to_keep_in_lib_dfs:
        if i in v:
            TempList = list(v[i])
            LibData_dict_reduced[k][i] = TempList

#Did above loop do what I thought it did?
print("Original dfs:")
for k, v in LibData_dict.items():
    print(k, v.shape)

print("\n", "Reduced dfs:")  
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

#Combine col names that mean same thing in different years into single 
Names_to_combine = ["ATTEND", "VISITS",  "C_LEGBAS", "C_LEGBASE", "DUPLI", "HRS_OPEN", "KIDATTEN", "KIDATTEND", 
                    "LIBRARIAN", "LIBRARIA", "POPU", "POPU_LSA", "POPU_UND", "POPU_UNDUP", "REFERENC", "REFERENCE", 
                    "STAFFEXP", "TOTEXP", "SUBSCRIP", "SUBSCRIPT", "TOTEXPCO", "TOTEXPCOL", 
                    "TOTOPEXP", "TOTOPEXP1", "TOTPEMP", "TOTSTAFF", "ZIP", "ZIP1"]

for i in range(0, len(Names_to_combine)-1, 2):
    for k, v in LibData_dict_reduced.items(): 
        if Names_to_combine[i] in v:
            name = Names_to_combine[i]
            newname = Names_to_combine[i] + "-" + Names_to_combine[i+1]
            v = v.rename(columns = {name: newname}, inplace = True)
        elif Names_to_combine[i+1] in v:
            name = Names_to_combine[i + 1]
            newname = Names_to_combine[i] + "-" + Names_to_combine[i+1]
            v = v.rename(columns = {name : newname}, inplace = True)

#Check that above code worked - names changes
for k, v in LibData_dict_reduced.items():
    print(v.columns)

for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

#Columns to add to DFs
"AUDIO is divided into AUDIO_PH, AUDIO_DL starting 2010. Source: 2010 Documentation."
"VIDEO was later split into VIDEO_DL and VIDEO_PH."

for k, v in LibData_dict_reduced.items(): 
    if "AUDIO_DL" in v:
        v["AUDIO"] = v['AUDIO_DL'] + v['AUDIO_PH']
    if "VIDEO_DL" in v:
        v["VIDEO"] = v['VIDEO_DL'] + v['VIDEO_PH']

#Check that # columns in some dataframes increased by 2 (expected)
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)
    
#Create DF w boolean values for prensence of each unique value in REDUCED dataframes
#THIS IS THE END GOAL OF THIS DATA

#Get all unique columns present in LibData_dict_reduced dataframes. 
ColumnsPresent_reduced = pd.DataFrame(index=range(0),columns=range(0))

for k, v in LibData_dict_reduced.items():
    dataframe_to_join_reduced = pd.DataFrame({k: list(v.columns)})
    ColumnsPresent_reduced = ColumnsPresent_reduced.join(dataframe_to_join_reduced, how = 'outer')

ColumnsPresent_reduced_str = ColumnsPresent_reduced.applymap(str)
ColumnsPresent_reduced_str = ColumnsPresent_reduced_str.reset_index(drop = True)
    
UniqueLabels_reduced = np.unique(ColumnsPresent_reduced_str.values)
LabelsBoolDF_reduced = pd.DataFrame(UniqueLabels_reduced)

#Create DF w boolean values for prensence of each unique value
#THIS IS THE END GOAL OF THIS DATA
for j in range(len(ColumnsPresent_reduced_str.columns)):
    lis = []
    for i in range(len(UniqueLabels_reduced)):
        lis.append(ColumnsPresent_reduced_str.iloc[:, j].isin([UniqueLabels_reduced[i]]).any())
    LabelsBoolDF_reduced[ColumnsPresent_reduced_str.columns[j]] = lis

## Sum all TRUE values. Value of 25 = all true
sum_list_reduced = []
for i in range(LabelsBoolDF_reduced.shape[0]):
    sum_list_reduced.append(sum(LabelsBoolDF_reduced.iloc[i, 1:26]))

#Label OK in Notes
LabelsBoolDF_reduced['TRUE SUM'] = sum_list_reduced
LabelsBoolDF_reduced['Notes'] = ""
LabelsBoolDF_reduced.loc[LabelsBoolDF_reduced[LabelsBoolDF_reduced['TRUE SUM']==25].index, 'Notes'] = "OK"

All_Cols_in_Reduced_dfs = list(LabelsBoolDF_reduced[0])
All_Cols_in_Reduced_dfs

for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)
    
#Delete columns deemed NOT useful for downstream analysis
NonUsefulColstoDelete = ['ENDDATE', 'LIBID', 'PHONE', 'POPU_UND-POPU_UNDUP', 'RSTATUS', 'TOTCOLL', 'ZIP-ZIP1']

LibData_dict_reduced_dropuseless = copy.deepcopy(LibData_dict_reduced)

for k, v in LibData_dict_reduced_dropuseless.items(): 
    for i in range(len(NonUsefulColstoDelete)):
        if NonUsefulColstoDelete[i] in v:
            v.drop([NonUsefulColstoDelete[i]], axis = 1, inplace = True)
            
#Check quickly whether columns were dropped
print("Reduced dfs:")
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

print("\n", "Reduced and dropped useless dfs:")
for k, v in LibData_dict_reduced_dropuseless.items():    
    print(k, v.shape)

#Get rid of negative values (replace w NaN)
#For all DFs, replace (-1, -3, -9) values with NaN. 
LibData_dict_reduced_dropuseless_NegRepWNaN = copy.deepcopy(LibData_dict_reduced_dropuseless)

for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    v[(v == -1) | (v == -3) | (v == -9)] = np.nan
    
#See if this worked (looks like it did)
print(sum(LibData_dict_reduced_dropuseless['_2014']['AUDIO_DL'] < 0))
print(np.where((LibData_dict_reduced_dropuseless['_2014']['AUDIO_DL'] < 0) == True))

print(sum(LibData_dict_reduced_dropuseless_NegRepWNaN['_2014']['AUDIO_DL'].isnull()))
print(np.where(LibData_dict_reduced_dropuseless_NegRepWNaN['_2014']['AUDIO_DL'].isnull()))


_2014 (9305, 159) <class 'pandas.core.frame.DataFrame'>
_2015 (9251, 159) <class 'pandas.core.frame.DataFrame'>
_2016 (9252, 151) <class 'pandas.core.frame.DataFrame'>
_2001 (9133, 108) <class 'pandas.core.frame.DataFrame'>
_2002 (9141, 107) <class 'pandas.core.frame.DataFrame'>
_2003 (9214, 110) <class 'pandas.core.frame.DataFrame'>
_2004 (9210, 128) <class 'pandas.core.frame.DataFrame'>
_2005 (9201, 131) <class 'pandas.core.frame.DataFrame'>
_2006 (9211, 139) <class 'pandas.core.frame.DataFrame'>
_2007 (9217, 148) <class 'pandas.core.frame.DataFrame'>
_2008 (9284, 150) <class 'pandas.core.frame.DataFrame'>
_2009 (9299, 152) <class 'pandas.core.frame.DataFrame'>
_2010 (9308, 154) <class 'pandas.core.frame.DataFrame'>
_2011 (9315, 157) <class 'pandas.core.frame.DataFrame'>
_2012 (9305, 155) <class 'pandas.core.frame.DataFrame'>
_2013 (9309, 157) <class 'pandas.core.frame.DataFrame'>
_2000 (9078, 108) <class 'pandas.core.frame.DataFrame'>
_1992 (8944, 83) <class 'pandas.core.frame.DataF

_2014 (9305, 69)
_2015 (9251, 72)
_2016 (9252, 77)
_2001 (9133, 52)
_2002 (9141, 52)
_2003 (9214, 53)
_2004 (9210, 56)
_2005 (9201, 60)
_2006 (9211, 61)
_2007 (9217, 61)
_2008 (9284, 62)
_2009 (9299, 64)
_2010 (9308, 67)
_2011 (9315, 67)
_2012 (9305, 67)
_2013 (9309, 68)
_2000 (9078, 52)
_1992 (8944, 44)
_1993 (8929, 44)
_1994 (8920, 44)
_1995 (8981, 49)
_1996 (8946, 49)
_1997 (8968, 49)
_1998 (8966, 51)
_1999 (9048, 52)
Reduced dfs:
_2014 (9305, 69)
_2015 (9251, 72)
_2016 (9252, 77)
_2001 (9133, 52)
_2002 (9141, 52)
_2003 (9214, 53)
_2004 (9210, 56)
_2005 (9201, 60)
_2006 (9211, 61)
_2007 (9217, 61)
_2008 (9284, 62)
_2009 (9299, 64)
_2010 (9308, 67)
_2011 (9315, 67)
_2012 (9305, 67)
_2013 (9309, 68)
_2000 (9078, 52)
_1992 (8944, 44)
_1993 (8929, 44)
_1994 (8920, 44)
_1995 (8981, 49)
_1996 (8946, 49)
_1997 (8968, 49)
_1998 (8966, 51)
_1999 (9048, 52)

 Reduced and dropped useless dfs:
_2014 (9305, 63)
_2015 (9251, 66)
_2016 (9252, 70)
_2001 (9133, 47)
_2002 (9141, 47)
_2003 (9214, 48)


In [123]:
#Reduce each yearly dataframe to FSCSKEY and usage data only
LibraryData_UsageOnly = dict()

for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    LibraryData_UsageOnly[k] = v[['FSCSKEY', 'POPU-POPU_LSA', 'ATTEND-VISITS']]

In [124]:
#Check that reduction worked
for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    print(k, v.shape, type(LibData_dict_reduced_dropuseless_NegRepWNaN[k]))
    print(k, LibraryData_UsageOnly[k].shape, type(LibraryData_UsageOnly[k]))

_2014 (9305, 63) <class 'pandas.core.frame.DataFrame'>
_2014 (9305, 3) <class 'pandas.core.frame.DataFrame'>
_2015 (9251, 66) <class 'pandas.core.frame.DataFrame'>
_2015 (9251, 3) <class 'pandas.core.frame.DataFrame'>
_2016 (9252, 70) <class 'pandas.core.frame.DataFrame'>
_2016 (9252, 3) <class 'pandas.core.frame.DataFrame'>
_2001 (9133, 47) <class 'pandas.core.frame.DataFrame'>
_2001 (9133, 3) <class 'pandas.core.frame.DataFrame'>
_2002 (9141, 47) <class 'pandas.core.frame.DataFrame'>
_2002 (9141, 3) <class 'pandas.core.frame.DataFrame'>
_2003 (9214, 48) <class 'pandas.core.frame.DataFrame'>
_2003 (9214, 3) <class 'pandas.core.frame.DataFrame'>
_2004 (9210, 50) <class 'pandas.core.frame.DataFrame'>
_2004 (9210, 3) <class 'pandas.core.frame.DataFrame'>
_2005 (9201, 54) <class 'pandas.core.frame.DataFrame'>
_2005 (9201, 3) <class 'pandas.core.frame.DataFrame'>
_2006 (9211, 55) <class 'pandas.core.frame.DataFrame'>
_2006 (9211, 3) <class 'pandas.core.frame.DataFrame'>
_2007 (9217, 55) <c

In [125]:
#Add usage column to each library
for k, v in LibraryData_UsageOnly.items():
    v['Usage'] = v['ATTEND-VISITS']/v['POPU-POPU_LSA']
    
#Check that column was added
for k, v in LibraryData_UsageOnly.items():
    print(k, v.shape)

//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


_2014 (9305, 4)
_2015 (9251, 4)
_2016 (9252, 4)
_2001 (9133, 4)
_2002 (9141, 4)
_2003 (9214, 4)
_2004 (9210, 4)
_2005 (9201, 4)
_2006 (9211, 4)
_2007 (9217, 4)
_2008 (9284, 4)
_2009 (9299, 4)
_2010 (9308, 4)
_2011 (9315, 4)
_2012 (9305, 4)
_2013 (9309, 4)
_2000 (9078, 4)
_1992 (8944, 4)
_1993 (8929, 4)
_1994 (8920, 4)
_1995 (8981, 4)
_1996 (8946, 4)
_1997 (8968, 4)
_1998 (8966, 4)
_1999 (9048, 4)


In [126]:
for k, v, in LibraryData_UsageOnly.items():
    newname = "Usage" + k
    v = v.rename(columns = {"Usage": newname}, inplace = True)

//anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [201]:
#MERGE ON FSCSKEY. 
LibraryData_UsageOnly_YearsMerged = pd.DataFrame()

for k, v in LibraryData_UsageOnly.items():
    if k == '_2014':
        LibraryData_UsageOnly_YearsMerged = v
    else: 
        LibraryData_UsageOnly_YearsMerged = pd.merge(LibraryData_UsageOnly_YearsMerged, v, how = 'outer', on = 'FSCSKEY')


In [202]:
#Keep usage only
LibraryData_UsageOnly_YearsMerged = LibraryData_UsageOnly_YearsMerged.drop(columns = ['POPU-POPU_LSA_x', 'ATTEND-VISITS_x', 
                                                                                      'POPU-POPU_LSA_y', 'ATTEND-VISITS_y',
                                                                                     'POPU-POPU_LSA', 'ATTEND-VISITS'])

LibraryData_UsageOnly_YearsMerged.shape


(10158, 26)

In [203]:
LibraryData_UsageOnly_YearsMerged

,FSCSKEY,Usage_2014,Usage_2015,Usage_2016,Usage_2001,Usage_2002,Usage_2003,Usage_2004,Usage_2005,Usage_2006,...,Usage_2013,Usage_2000,Usage_1992,Usage_1993,Usage_1994,Usage_1995,Usage_1996,Usage_1997,Usage_1998,Usage_1999
0,AK0001,2.612433,2.873905,3.269212,1.586265,3.452769,3.444688,2.511161,2.603282,2.745424,...,4.591377,0.836397,1.840943,1.835536,1.673360,1.518761,1.595745,1.705566,1.596071,1.211584
1,AK0002,3.137169,3.055469,2.878410,3.200255,3.155238,3.158349,3.470295,3.472364,3.354100,...,2.670980,3.041524,3.675581,2.097850,1.649358,3.585049,3.634552,3.626281,3.385498,3.417765
2,AK0003,4.181818,3.152074,4.810924,7.528610,7.139276,8.531915,9.476744,10.635714,11.931900,...,4.468085,8.514986,3.491525,3.950769,9.403077,4.504792,6.740675,6.375221,6.797710,6.045802
3,AK0006,7.207178,7.395649,7.170884,4.387498,3.568689,3.984404,3.772079,2.931544,2.952512,...,4.973877,3.304697,6.573541,5.200080,5.989219,10.587103,9.023306,10.670748,9.728334,2.094087
4,AK0007,3.383491,4.705518,6.071060,3.970386,3.993161,3.222895,4.713906,4.867173,5.071891,...,3.254871,3.657753,3.796752,3.194175,2.661563,2.533226,2.552051,3.020296,3.203645,3.676684
5,AK0008,16.483516,16.756757,16.186869,9.211712,9.333333,10.619469,11.636364,8.564220,9.289216,...,14.795918,11.261261,12.244898,12.748299,13.194444,12.068966,5.814815,5.729167,10.120482,10.240964
6,AK0011,21.437500,3.979592,7.326087,75.780000,35.714286,24.081633,12.745098,22.115385,21.545455,...,14.250000,51.460000,13.360996,14.107884,45.454545,45.783133,42.666667,8.108108,7.333333,21.666667
7,AK0014,17.977528,12.307692,16.881356,35.045455,10.913793,19.357895,21.202247,18.764045,14.080460,...,18.729412,23.295455,18.467153,19.576642,20.641667,14.463636,22.678082,24.908333,26.970874,25.728395
8,AK0015,11.352542,13.402214,11.060000,5.524011,11.733333,11.826816,11.925926,12.136628,10.582633,...,12.985663,5.395994,4.128472,4.336918,4.595149,3.844732,4.467480,4.462527,4.828753,5.013699
9,AK0016,10.025372,8.641965,7.916415,7.109617,8.910025,9.156408,11.854656,14.948864,12.555857,...,9.858384,7.519152,9.174121,7.835383,9.442049,6.929907,7.113126,7.314957,10.112797,7.261766


In [204]:
#Reorganize columns by date
col_names = list(LibraryData_UsageOnly_YearsMerged.columns)
col_order = col_names[0:1] + col_names[18:26] + col_names[17:18] + col_names[4:17] + col_names[1:4]
LibraryData_UsageOnly_YearsMerged = LibraryData_UsageOnly_YearsMerged[col_order]

In [211]:
#Get usage for window. Divide by number of values available (of 4) to correct for missing values
#Good news is, if dividing by 0 (i.e. all 5 NaN), usage is NaN. 
LibraryData_UsageOnly_YearsMerged["Window1"] = (LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_1992':'Usage_1996'].sum(axis = 1))/(5-LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_1992':'Usage_1996'].isnull().sum(axis = 1))
LibraryData_UsageOnly_YearsMerged["Window2"] = (LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_1997':'Usage_2001'].sum(axis = 1))/(5-LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_1997':'Usage_2001'].isnull().sum(axis = 1))
LibraryData_UsageOnly_YearsMerged["Window3"] = (LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_2002':'Usage_2006'].sum(axis = 1))/(5-LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_2002':'Usage_2006'].isnull().sum(axis = 1))
LibraryData_UsageOnly_YearsMerged["Window4"] = (LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_2007':'Usage_2011'].sum(axis = 1))/(5-LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_2007':'Usage_2011'].isnull().sum(axis = 1))
LibraryData_UsageOnly_YearsMerged["Window5"] = (LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_2012':'Usage_2016'].sum(axis = 1))/(5-LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_2012':'Usage_2016'].isnull().sum(axis = 1))


//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [221]:
LibraryData_WindowUsage = LibraryData_UsageOnly_YearsMerged[["FSCSKEY", "Window1", "Window2", "Window3", "Window4", "Window5"]]

In [236]:
#Get index for all libaries where usage data does not exist. 
IndexNaNForUsageForAtLeastOneWindow = LibraryData_WindowUsage[LibraryData_WindowUsage.Window1.isnull() | 
                                                              LibraryData_WindowUsage.Window2.isnull() | 
                                                              LibraryData_WindowUsage.Window3.isnull() | 
                                                              LibraryData_WindowUsage.Window4.isnull() | 
                                                              LibraryData_WindowUsage.Window5.isnull()].index
print(len(IndexNaNForUsageForAtLeastOneWindow))

1572


In [237]:
#Remove all rows (libraries) for which Usage data missing for any window!
LibraryData_WindowUsage_Minus1572 = LibraryData_WindowUsage.drop(IndexNaNForUsageForAtLeastOneWindow, axis=0)
print(LibraryData_WindowUsage.shape, LibraryData_WindowUsage_Minus1572.shape)

(10158, 6) (8586, 6)


In [238]:
LibraryData_WindowUsage_Minus1572

,FSCSKEY,Window1,Window2,Window3,Window4,Window5
0,AK0001,1.692869,1.387176,2.951465,4.108389,3.522798
1,AK0002,2.928478,3.334265,3.322069,2.854253,2.843573
2,AK0003,5.618168,7.052466,9.543110,7.727554,4.265914
3,AK0006,7.474650,6.037073,3.441846,3.088870,6.132112
4,AK0007,2.947553,3.505753,4.373805,3.980720,4.308130
5,AK0008,11.214284,9.312717,9.888520,10.613678,15.618525
6,AK0011,32.274645,32.869622,23.240371,14.583177,13.762272
7,AK0014,19.165436,27.189702,16.863688,16.337115,16.497565
8,AK0015,4.274550,5.044997,11.641067,10.047938,11.949845
9,AK0016,8.098917,7.863658,11.485162,11.323616,9.624783


In [256]:
#Now, let's work on feature DFs

#Delete unnecessary columns from all DFs.
#These are things that don't change over time and are not numeric!
#i.e. Library cannot change whether it's rural or in city, so delete those columns!

Dict_all_dfs = copy.deepcopy(LibData_dict_reduced_dropuseless_NegRepWNaN)
Colstodelete = ['ADDRESS', 'CITY', 'LIBNAME', 'C_LEGBAS-C_LEGBASE', 'STABR', 'GEOCODE', 'CNTY']

for k, v in Dict_all_dfs.items():
    for i in range(len(Colstodelete)):
        if Colstodelete[i] in v:
            v.drop(Colstodelete[i], axis=1, inplace = True)

In [263]:
#Check that deletion worked. 
print("OldDFCOLUMNS:")
for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    print(k, v.columns)
print("NewDFCOLUMNS:")
for k, v in Dict_all_dfs.items():
    print(k, v.columns)

OldDFCOLUMNS:
_2014 Index(['ADDRESS', 'AUDIO_DL', 'AUDIO_PH', 'BENEFIT', 'BKMOB', 'BKVOL',
       'BRANLIB', 'CAPITAL', 'CAP_REV', 'CENTLIB', 'CITY', 'CNTY',
       'C_LEGBAS-C_LEGBASE', 'EBOOK', 'ELMATCIR', 'ELMATEXP', 'FCAP_REV',
       'FSCSKEY', 'GEOCODE', 'GPTERMS', 'DUPLI-HRS_OPEN', 'KIDATTEN-KIDATTEND',
       'KIDCIRCL', 'KIDPRO', 'LIBNAME', 'LIBRARIAN-LIBRARIA', 'LOANFM',
       'LOANTO', 'LOCALE', 'LOCGVT', 'MASTER', 'OBEREG', 'OCAP_REV', 'OTHINCM',
       'OTHMATEX', 'OTHOPEXP', 'OTHPAID', 'PITUSR', 'POPU-POPU_LSA',
       'PRMATEXP', 'REFERENC-REFERENCE', 'REGBOR', 'SALARIES', 'SCAP_REV',
       'STABR', 'STAFFEXP-TOTEXP', 'STGVT', 'SUBSCRIP-SUBSCRIPT', 'TOTATTEN',
       'TOTCIR', 'TOTEXPCO-TOTEXPCOL', 'TOTINCM', 'TOTOPEXP-TOTOPEXP1',
       'TOTPRO', 'TOTPEMP-TOTSTAFF', 'VIDEO_DL', 'VIDEO_PH', 'ATTEND-VISITS',
       'WIFISESS', 'YAATTEN', 'YAPRO', 'AUDIO', 'VIDEO'],
      dtype='object')
_2015 Index(['ADDRESS', 'AUDIO_DL', 'AUDIO_PH', 'BENEFIT', 'BKMOB', 'BKVOL',
       

In [269]:
list_of_all_col_names = list()

for k, v in Dict_all_dfs.items():
    list_of_all_col_names.extend(v.columns)

Unique_col_names = set(list_of_all_col_names)
print(len(Unique_col_names))

69


In [335]:
#Make sure each df has all 69 columns. If column did not exist, add this column with NaN
Unique_col_names_list = list(Unique_col_names)

Dict_all_dfs_69cols = copy.deepcopy(Dict_all_dfs)

for k, v in Dict_all_dfs_69cols.items():
    for i in range(len(Unique_col_names_list)):
        if Unique_col_names_list[i] not in v:
            v[Unique_col_names_list[i]] = np.nan

In [336]:
#Make sure all dfs now have 69 columns
for k, v in Dict_all_dfs_69cols.items():
    print(k, v.shape)

_2014 (9305, 69)
_2015 (9251, 69)
_2016 (9252, 69)
_2001 (9133, 69)
_2002 (9141, 69)
_2003 (9214, 69)
_2004 (9210, 69)
_2005 (9201, 69)
_2006 (9211, 69)
_2007 (9217, 69)
_2008 (9284, 69)
_2009 (9299, 69)
_2010 (9308, 69)
_2011 (9315, 69)
_2012 (9305, 69)
_2013 (9309, 69)
_2000 (9078, 69)
_1992 (8944, 69)
_1993 (8929, 69)
_1994 (8920, 69)
_1995 (8981, 69)
_1996 (8946, 69)
_1997 (8968, 69)
_1998 (8966, 69)
_1999 (9048, 69)


In [298]:
#BELOW, I have a better way to do this. This code takes way too long and makes a huge dataframe.
newdf = pd.DataFrame()

for i in range(len(Unique_col_names_list)):
    for k, v in Dict_all_dfs_69cols.items():
        newdf = pd.concat([newdf, v[['FSCSKEY', Unique_col_names_list[i]]]], axis = 1)

In [337]:
#NEED to get rid of FSCSKEY in Unique_col_names_list!!
print(len(Unique_col_names_list))
if 'FSCSKEY' in Unique_col_names_list: Unique_col_names_list.remove('FSCSKEY')
print(len(Unique_col_names_list))

69
68


In [339]:
#Make dataframes of each feature by year
list_of_dataframes = list()
newdict = dict()

for i in range(len(Unique_col_names_list)):
    tempdf = pd.DataFrame()
    for k, v in Dict_all_dfs_69cols.items():
        newdict[k] = v[['FSCSKEY', Unique_col_names_list[i]]]
    for k, v in newdict.items():
        if k == '_2014':
            tempdf = v
        else: 
            tempdf = pd.merge(tempdf, v, how = 'outer', on = 'FSCSKEY')
    list_of_dataframes.append(tempdf)

In [342]:
#Check that this worked
for dataframe in list_of_dataframes:
    print(dataframe.shape)

(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)
(10158, 26)


In [354]:
YearOrder = ['_2014', '_2015', '_2016', '_2001', '_2002', '_2003', 
              '_2004','_2005', '_2006', '_2007', '_2008', '_2009', 
              '_2010', '_2011', '_2012', '_2013', '_2000', '_1992', 
              '_1993', '_1994', '_1995', '_1996', '_1997', '_1998', '_1999']

In [395]:
#Add year to dataframes
list_of_dataframe_with_years = copy.deepcopy(list_of_dataframes)

for dataf in list_of_dataframe_with_years:
    colnamelist = ['FSCSKEY']
    for i in range(len(YearOrder)):
        colnamelist.append(str(dataf.columns[25] + YearOrder[i]))
    dataf.columns = colnamelist

In [418]:
#Rearrange columns by date
list_of_dataframe_with_years_ordered = copy.deepcopy(list_of_dataframe_with_years)
list_of_dataframe_with_years_ordered_2 = list()

for dataf in list_of_dataframe_with_years_ordered:
    colnames = dataf.columns.tolist()
    colnames = colnames[0:1] + colnames[18:26] + colnames[17:18] + colnames[4:17] + colnames[1:4]
    dataf = dataf[colnames]
    list_of_dataframe_with_years_ordered_2.append(dataf)

In [436]:
List_of_dataframe_w_year_Windows = copy.deepcopy(list_of_dataframe_with_years_ordered_2)
for dataf in List_of_dataframe_w_year_Windows:
    name = dataf.columns[2][:-5]
    dataf[name +"_Window1"] = (dataf.iloc[:, 1:6].sum(axis = 1))/(5-dataf.iloc[:, 1:6].isnull().sum(axis = 1))
    dataf[name +"_Window2"] = (dataf.iloc[:, 6:11].sum(axis = 1))/(5-dataf.iloc[:, 6:11].isnull().sum(axis = 1))
    dataf[name +"_Window3"] = (dataf.iloc[:, 11:16].sum(axis = 1))/(5-dataf.iloc[:, 11:16].isnull().sum(axis = 1))
    dataf[name +"_Window4"] = (dataf.iloc[:, 16:21].sum(axis = 1))/(5-dataf.iloc[:, 16:21].isnull().sum(axis = 1))
    dataf[name +"_Window5"] = (dataf.iloc[:, 21:26].sum(axis = 1))/(5-dataf.iloc[:, 21:26].isnull().sum(axis = 1))

In [470]:
#List of dataframes for each 
List_of_dataframe_WindowsONLY = copy.deepcopy(List_of_dataframe_w_year_Windows)
List_of_dataframe_WindowsONLY_2 = list()

for dataf in List_of_dataframe_WindowsONLY:
    dataf = dataf.drop(dataf.columns[1:26], axis = 1)
    List_of_dataframe_WindowsONLY_2.append(dataf)

In [481]:
#Combine windowed DFs with Windowed USAGE data.
#THIS IS A DATAFRAME I MAY WANT TO GO BACK TO: WINDOWS FOR USAGE AND ALL COLUMNS.
CombinedWindowedDF = copy.deepcopy(LibraryData_WindowUsage_Minus1572)

for dataf in List_of_dataframe_WindowsONLY_2:
    CombinedWindowedDF = pd.merge(CombinedWindowedDF, dataf, how = "left", on = 'FSCSKEY')
    
print(LibraryData_WindowUsage_Minus1572.shape, CombinedWindowedDF.shape)   

(8586, 6) (8586, 346)


In [488]:
CombinedWindowedDF = CombinedWindowedDF.append(CombinedWindowedDF.isna().sum(),ignore_index=True)
print(CombinedWindowedDF.shape)

(8587, 346)


In [505]:
CombinedWindowedDF.iloc[8586, 6] == 8586

True

In [511]:
#SAVE DATAFRAME with features ONLY where no window is missing all values. 
CombinedWindowedDF_DELETEAll5WINDOWSIFATLEAST1WINDOWHAS8586 = pd.DataFrame()

for i in range(1, CombinedWindowedDF.shape[1], 5):
    if ((CombinedWindowedDF.iloc[8586, i] == 8586) | (CombinedWindowedDF.iloc[8586, (i+1)] == 8586) | (CombinedWindowedDF.iloc[8586, (i+2)] == 8586) |
        (CombinedWindowedDF.iloc[8586, (i+3)] == 8586) | (CombinedWindowedDF.iloc[8586, (i+4)] == 8586)):
    else: 
        CombinedWindowedDF_DELETEAll5WINDOWSIFATLEAST1WINDOWHAS8586 = pd.concat([CombinedWindowedDF_DELETEAll5WINDOWSIFATLEAST1WINDOWHAS8586, CombinedWindowedDF.iloc[:, i:i+5]], axis=1, sort=False)
        
        

Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying
Not copying


In [514]:
print(CombinedWindowedDF_DELETEAll5WINDOWSIFATLEAST1WINDOWHAS8586.columns.tolist())

['Window1', 'Window2', 'Window3', 'Window4', 'Window5', 'AUDIO_Window1', 'AUDIO_Window2', 'AUDIO_Window3', 'AUDIO_Window4', 'AUDIO_Window5', 'BRANLIB_Window1', 'BRANLIB_Window2', 'BRANLIB_Window3', 'BRANLIB_Window4', 'BRANLIB_Window5', 'LOANTO_Window1', 'LOANTO_Window2', 'LOANTO_Window3', 'LOANTO_Window4', 'LOANTO_Window5', 'TOTCIR_Window1', 'TOTCIR_Window2', 'TOTCIR_Window3', 'TOTCIR_Window4', 'TOTCIR_Window5', 'SALARIES_Window1', 'SALARIES_Window2', 'SALARIES_Window3', 'SALARIES_Window4', 'SALARIES_Window5', 'BKVOL_Window1', 'BKVOL_Window2', 'BKVOL_Window3', 'BKVOL_Window4', 'BKVOL_Window5', 'LOCGVT_Window1', 'LOCGVT_Window2', 'LOCGVT_Window3', 'LOCGVT_Window4', 'LOCGVT_Window5', 'BKMOB_Window1', 'BKMOB_Window2', 'BKMOB_Window3', 'BKMOB_Window4', 'BKMOB_Window5', 'OTHINCM_Window1', 'OTHINCM_Window2', 'OTHINCM_Window3', 'OTHINCM_Window4', 'OTHINCM_Window5', 'OTHPAID_Window1', 'OTHPAID_Window2', 'OTHPAID_Window3', 'OTHPAID_Window4', 'OTHPAID_Window5', 'LOANFM_Window1', 'LOANFM_Window2'

In [568]:
newdf = pd.DataFrame()
dftoappendto = pd.DataFrame()

list_of_dataframes = list()

for i in range(CombinedWindowedDF_DELETEAll5WINDOWSIFATLEAST1WINDOWHAS8586.shape[0]):
    for j in range(0, CombinedWindowedDF_DELETEAll5WINDOWSIFATLEAST1WINDOWHAS8586.shape[1], 5):
        name = CombinedWindowedDF_DELETEAll5WINDOWSIFATLEAST1WINDOWHAS8586.columns[j]
        newdf[name] = list(CombinedWindowedDF_DELETEAll5WINDOWSIFATLEAST1WINDOWHAS8586.iloc[i, j:j+5])
    list_of_dataframes.append(newdf)
    newdf = pd.DataFrame()


In [668]:
#Get final dataframe with Usage and all features kept. 
Temporaldataframe = pd.concat(list_of_dataframes)

In [669]:
#Re-set index of dataframe
Temporaldataframe = Temporaldataframe.reset_index(drop=True)

In [670]:
#NOW NEED TO CALC CHANGE IN USAGE OR CHANGE IN VISITS
#Rename col 0 usage. 
Temporaldataframe.rename(columns = {'Window1':'Usage' }, inplace = True)

print(Temporaldataframe.columns)

Index(['Usage', 'AUDIO_Window1', 'BRANLIB_Window1', 'LOANTO_Window1',
       'TOTCIR_Window1', 'SALARIES_Window1', 'BKVOL_Window1', 'LOCGVT_Window1',
       'BKMOB_Window1', 'OTHINCM_Window1', 'OTHPAID_Window1', 'LOANFM_Window1',
       'REFERENC-REFERENCE_Window1', 'ATTEND-VISITS_Window1',
       'SUBSCRIP-SUBSCRIPT_Window1', 'TOTOPEXP-TOTOPEXP1_Window1',
       'LIBRARIAN-LIBRARIA_Window1', 'TOTEXPCO-TOTEXPCOL_Window1',
       'OBEREG_Window1', 'KIDCIRCL_Window1', 'STAFFEXP-TOTEXP_Window1',
       'KIDATTEN-KIDATTEND_Window1', 'STGVT_Window1',
       'TOTPEMP-TOTSTAFF_Window1', 'VIDEO_Window1', 'CENTLIB_Window1',
       'BENEFIT_Window1', 'TOTINCM_Window1', 'MASTER_Window1',
       'CAPITAL_Window1', 'DUPLI-HRS_OPEN_Window1', 'ELMATEXP_Window1',
       'POPU-POPU_LSA_Window1', 'OTHOPEXP_Window1'],
      dtype='object')


In [671]:
#Move ATTEND-VISITS_Window1 to second column
cols = Temporaldataframe.columns.tolist()
cols_rearr = cols[0:1] + cols[13:14] + cols[1:13] + cols[14:len(cols)]
print(len(cols_rearr), len(cols))

Temporaldataframe = Temporaldataframe[cols_rearr]

34 34


In [672]:
Temporaldataframe.insert(1, "Usage difference", "")
Temporaldataframe.insert(3, "VISIT difference", "")

In [676]:
for i in range(Temporaldataframe.shape[0]-1):
    Temporaldataframe.loc[i, 'Usage difference'] = (Temporaldataframe.loc[i+1, 'Usage']-Temporaldataframe.loc[i, 'Usage'])/Temporaldataframe.loc[i, 'Usage']
    Temporaldataframe.loc[i, 'VISIT difference'] = (Temporaldataframe.loc[i+1, 'ATTEND-VISITS_Window1']-Temporaldataframe.loc[i, 'ATTEND-VISITS_Window1'])/Temporaldataframe.loc[i, 'ATTEND-VISITS_Window1']
    

//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [693]:
#Re-set index of dataframe just in case
Temporaldataframe = Temporaldataframe.reset_index(drop=True)

#delete every 5th row because this is not predictive...
every5 = list(np.arange(4,42935,5))
Temporaldataframe_deleteevery5throw = Temporaldataframe.drop(Temporaldataframe.index[[every5]])

In [696]:
Temporaldataframe_deleteevery5throw

,Usage,Usage difference,ATTEND-VISITS_Window1,VISIT difference,AUDIO_Window1,BRANLIB_Window1,LOANTO_Window1,TOTCIR_Window1,SALARIES_Window1,BKVOL_Window1,...,VIDEO_Window1,CENTLIB_Window1,BENEFIT_Window1,TOTINCM_Window1,MASTER_Window1,CAPITAL_Window1,DUPLI-HRS_OPEN_Window1,ELMATEXP_Window1,POPU-POPU_LSA_Window1,OTHOPEXP_Window1
0,1.692869,-0.180577,2550.0,0.0280784,27.2,0.0,5.4,7416.0,NaN,9982.0,...,407.6,1.0,NaN,9817.0,0.000,0.0,520.0,0.0,1517.0,NaN
1,1.387176,1.12768,2621.6,1.03692,54.6,0.0,80.2,6399.2,NaN,11509.0,...,246.4,1.0,NaN,11765.8,0.000,0.0,572.0,140.0,1911.2,NaN
2,2.951465,0.391983,5340.0,0.437566,156.6,0.0,1.4,8693.0,NaN,12079.6,...,933.2,1.0,NaN,13343.0,0.000,0.0,1144.0,38.0,1806.0,NaN
3,4.108389,-0.142535,7676.6,-0.0647943,213.0,0.0,0.0,15732.4,NaN,13279.8,...,3261.4,1.0,NaN,24208.2,0.000,0.0,1404.0,255.6,1869.4,NaN
5,2.928478,0.138566,762638.8,0.130114,38998.2,4.4,4748.0,1298131.6,3363396.40,445421.4,...,3176.8,1.0,1319309.20,7901173.0,35.470,0.0,9720.6,71423.5,259398.0,2435451.80
6,3.334265,-0.00365759,861869.0,0.0656176,24529.4,5.0,6267.2,1249519.4,3860827.60,548607.2,...,11337.2,1.0,1344598.80,9221136.0,40.810,0.0,10837.8,103201.6,258595.8,2962021.80
7,3.322069,-0.140821,918422.8,-0.101096,30163.0,5.0,7772.8,1408061.0,3682900.80,557654.0,...,27497.8,1.0,1554931.00,8767147.8,30.024,10478.4,11480.8,83062.6,276325.0,2658881.20
8,2.854253,-0.00374197,825573.6,0.0322847,33920.6,4.2,7054.4,1565763.6,3807724.20,615622.0,...,46988.2,1.0,2623292.00,11274406.6,24.816,1181482.0,12076.2,91409.8,289485.6,3796511.40
10,5.618168,0.255296,3512.2,-0.0763624,405.8,0.0,23.6,3039.6,NaN,11538.0,...,636.6,1.0,NaN,14254.8,0.000,0.0,512.8,NaN,627.6,NaN
11,7.052466,0.353159,3244.0,-0.0543773,463.8,0.0,2.4,3070.4,NaN,13433.0,...,1097.2,1.0,NaN,13120.0,0.000,0.0,510.0,119.6,469.4,NaN


In [700]:
#Reset index 
Temporaldataframe_deleteevery5throw = Temporaldataframe_deleteevery5throw.reset_index(drop=True)

print(Temporaldataframe_deleteevery5throw.shape)
#How many rows have NaN in them?
#Two different ways to ask this. Second way gives index.
print(sum(pd.isnull(Temporaldataframe_deleteevery5throw).any(axis=1)))
print(len(pd.isnull(Temporaldataframe_deleteevery5throw).any(1).nonzero()[0]))

(34348, 36)
12369
12369


In [701]:
#Remove ALL NaNs. I am cutting corners here...
NaNIndex = pd.isnull(Temporaldataframe_deleteevery5throw).any(1).nonzero()[0]
Temporaldataframe_deleteevery5throw_noNaN = Temporaldataframe_deleteevery5throw.drop(NaNIndex)

print(Temporaldataframe_deleteevery5throw.shape)
print(Temporaldataframe_deleteevery5throw_noNaN.shape)



(34348, 36)
(21979, 36)


In [703]:
Temporaldataframe_deleteevery5throw_noNaN.columns

Index(['Usage', 'Usage difference', 'ATTEND-VISITS_Window1',
       'VISIT difference', 'AUDIO_Window1', 'BRANLIB_Window1',
       'LOANTO_Window1', 'TOTCIR_Window1', 'SALARIES_Window1', 'BKVOL_Window1',
       'LOCGVT_Window1', 'BKMOB_Window1', 'OTHINCM_Window1', 'OTHPAID_Window1',
       'LOANFM_Window1', 'REFERENC-REFERENCE_Window1',
       'SUBSCRIP-SUBSCRIPT_Window1', 'TOTOPEXP-TOTOPEXP1_Window1',
       'LIBRARIAN-LIBRARIA_Window1', 'TOTEXPCO-TOTEXPCOL_Window1',
       'OBEREG_Window1', 'KIDCIRCL_Window1', 'STAFFEXP-TOTEXP_Window1',
       'KIDATTEN-KIDATTEND_Window1', 'STGVT_Window1',
       'TOTPEMP-TOTSTAFF_Window1', 'VIDEO_Window1', 'CENTLIB_Window1',
       'BENEFIT_Window1', 'TOTINCM_Window1', 'MASTER_Window1',
       'CAPITAL_Window1', 'DUPLI-HRS_OPEN_Window1', 'ELMATEXP_Window1',
       'POPU-POPU_LSA_Window1', 'OTHOPEXP_Window1'],
      dtype='object')

In [705]:
import matplotlib.pylab as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [739]:
#df.index[np.isinf(df).any(1)]
#Temporaldataframe_deleteevery5throw_noNaN.index[np.isinf(Temporaldataframe_deleteevery5throw_noNaN).any(1)]


#Temporaldataframe_deleteevery5throw_noNaN[~Temporaldataframe_deleteevery5throw_noNaN.isin([np.inf]).any(1)]


#print(sum(pd.isinf(Temporaldataframe_deleteevery5throw_noNaN).any(axis=1)))
#print(len(pd.isnull(Temporaldataframe_deleteevery5throw_noNaN).any(1).nonzero()[0]))

#np.isnan(X).any()
#np.isnan(y).any()



AUDIO_Window1                 False
BRANLIB_Window1               False
LOANTO_Window1                False
TOTCIR_Window1                False
SALARIES_Window1              False
BKVOL_Window1                 False
LOCGVT_Window1                False
BKMOB_Window1                 False
OTHINCM_Window1               False
OTHPAID_Window1               False
LOANFM_Window1                False
REFERENC-REFERENCE_Window1    False
SUBSCRIP-SUBSCRIPT_Window1    False
TOTOPEXP-TOTOPEXP1_Window1    False
LIBRARIAN-LIBRARIA_Window1    False
TOTEXPCO-TOTEXPCOL_Window1    False
OBEREG_Window1                False
KIDCIRCL_Window1              False
STAFFEXP-TOTEXP_Window1       False
KIDATTEN-KIDATTEND_Window1    False
STGVT_Window1                 False
TOTPEMP-TOTSTAFF_Window1      False
VIDEO_Window1                 False
CENTLIB_Window1               False
BENEFIT_Window1               False
TOTINCM_Window1               False
MASTER_Window1                False
CAPITAL_Window1             

In [735]:
#Try to predict visit difference!
#THIS GIVES ERROR b/c y has two infinite values!
X = Temporaldataframe_deleteevery5throw_noNaN.iloc[:, 4:].astype(float)
y = Temporaldataframe_deleteevery5throw_noNaN.iloc[:, 3].astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

#Try to predict usage difference!


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [757]:
#y contains infinite value
print(np.isinf(X).any())
print(np.isinf(y).any())

print(np.where(np.isinf(y)))

#y has two infinite values.

y[16554:16558]

AUDIO_Window1                 False
BRANLIB_Window1               False
LOANTO_Window1                False
TOTCIR_Window1                False
SALARIES_Window1              False
BKVOL_Window1                 False
LOCGVT_Window1                False
BKMOB_Window1                 False
OTHINCM_Window1               False
OTHPAID_Window1               False
LOANFM_Window1                False
REFERENC-REFERENCE_Window1    False
SUBSCRIP-SUBSCRIPT_Window1    False
TOTOPEXP-TOTOPEXP1_Window1    False
LIBRARIAN-LIBRARIA_Window1    False
TOTEXPCO-TOTEXPCOL_Window1    False
OBEREG_Window1                False
KIDCIRCL_Window1              False
STAFFEXP-TOTEXP_Window1       False
KIDATTEN-KIDATTEND_Window1    False
STGVT_Window1                 False
TOTPEMP-TOTSTAFF_Window1      False
VIDEO_Window1                 False
CENTLIB_Window1               False
BENEFIT_Window1               False
TOTINCM_Window1               False
MASTER_Window1                False
CAPITAL_Window1             

25824         inf
25825    0.623438
25826   -1.000000
25827         inf
Name: VISIT difference, dtype: float64

In [762]:
#Here are those two infinite values.
Temporaldataframe_deleteevery5throw_noNaN = Temporaldataframe_deleteevery5throw_noNaN.reset_index(drop = True)
Temporaldataframe_deleteevery5throw_noNaN.loc[16554:16558, :]

,Usage,Usage difference,ATTEND-VISITS_Window1,VISIT difference,AUDIO_Window1,BRANLIB_Window1,LOANTO_Window1,TOTCIR_Window1,SALARIES_Window1,BKVOL_Window1,...,VIDEO_Window1,CENTLIB_Window1,BENEFIT_Window1,TOTINCM_Window1,MASTER_Window1,CAPITAL_Window1,DUPLI-HRS_OPEN_Window1,ELMATEXP_Window1,POPU-POPU_LSA_Window1,OTHOPEXP_Window1
16554,0.000000,inf,0.0,inf,0.0,0.0,25.4,20160.8,73556.6,1402.2,...,0.0,0.6,29152.6,178076.6,0.0,1818.2,452.666667,151.0,1359.0,42440.8
16555,0.850498,1.43647,1024.0,0.623438,9.0,0.0,213.8,19665.4,89539.6,3713.2,...,9.4,0.4,32519.0,264458.8,0.0,10212.2,453.600000,2239.0,1061.6,101259.4
16556,2.072215,-1,1662.4,-1,106.8,0.0,166.8,17368.8,138335.8,3530.6,...,291.6,0.4,65501.0,338383.6,0.0,32510.8,2152.200000,6089.2,1048.8,110340.0
16557,0.000000,inf,0.0,inf,5080.0,0.0,317.6,10138.2,173902.0,4292.6,...,1649.0,0.4,90969.6,453679.2,0.0,13780.0,2006.400000,19881.8,495.0,135481.4
16558,3.677209,0.36876,22365.6,0.435195,357.6,0.0,416.4,66443.6,86581.8,18452.0,...,472.6,1.0,19230.6,160007.8,0.0,5810.8,2385.000000,1017.0,6090.6,19782.8


In [774]:
#Try to predict usage difference!
XYFrame = Temporaldataframe_deleteevery5throw_noNaN.drop([16554,16557])

X = XYFrame.iloc[:, 4:].astype(float)
y = XYFrame.iloc[:, 3].astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

model.coef_

0.006693795562794591


array([-6.12850845e-07,  1.16584030e-03, -1.49117367e-06, -7.75272129e-09,
        1.36971531e-06, -8.50679307e-08, -2.30092248e-07, -3.41187695e-03,
       -2.37986688e-07, -5.39559613e-02,  1.11191635e-07, -4.01151410e-09,
       -1.17910068e-05, -4.21240078e-06, -5.50015329e-02,  4.32722926e-06,
        6.95311841e-04, -2.14865660e-08,  2.79020147e-06, -7.55747389e-07,
       -2.15827272e-07,  5.45462950e-02, -2.58470510e-06, -9.12467042e-02,
        1.42048567e-06,  2.44699597e-07,  2.51499099e-03,  3.06171265e-08,
        9.70272209e-08, -3.06720702e-07,  5.98213890e-08,  4.22298786e-06])

In [766]:
print(Temporaldataframe_deleteevery5throw_noNaN.shape)
print(XYFrame.shape)

(21979, 36)
(21977, 36)
